In [1]:
import numpy as np
from glob import glob
import cv2, os, random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
path = 'dogs-vs-cats/train'

In [4]:
ROW, COL = 96, 96

dogs, cats = [], []
y_dogs, y_cats = [], []

In [5]:
def load_dogs():
    print('Loading all dog images\n')
    dog_path = os.path.join(path, 'dog*')
    for dog_img in glob(dog_path):
        dog = cv2.imread(dog_img)
        dog = cv2.cvtColor(dog, cv2.COLOR_BGR2GRAY)
        dog = cv2.resize(dog, (ROW, COL))
        dog = image.img_to_array(dog)
        dogs.append(dog)
    print('All dog images loaded')

In [6]:
def load_cats():
    print('Loading all dog images\n')
    cat_path = os.path.join(path, 'cat*')
    for cat_img in glob(cat_path):
        cat = cv2.imread(cat_img)
        cat = cv2.cvtColor(cat, cv2.COLOR_BGR2GRAY)
        cat = cv2.resize(cat, (ROW, COL))
        cat = image.img_to_array(cat)
        cats.append(cat)
    print('All cat images loaded')

In [7]:
load_dogs()

Loading all dog images

All dog images loaded


In [8]:
load_cats()

Loading all dog images

All cat images loaded


In [9]:
classes = ['dog', 'cat']

In [10]:
def show_dogs():
    plt.figure(figsize=(12,8))
    for i in range(5):
        plt.subplot(1, 5, i+1)
        img = image.array_to_imgay_to_img(random.choice(dogs))
        plt.imshow(img)
        
        plt.axis('off')
        plt.title('Supposed to be a {}'.format(classes[0]))
    plt.show()

In [11]:
def show_cats():
    plt.figure(figsize=(12,8))
    for i in range(5):
        plt.subplot(1,5, i+1)
        img = image.array_to_img(random.choice(cats))
        plt.imshow(img)
        
        plt.axis('off')
        plt.title('Supposed to be a {}'.format(classes[1]))
    plt.show()

In [12]:
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)

In [13]:
y_dogs = [1 for item in enumerate(dogs)]
y_cats = [0 for item in enumerate(cats)]

In [14]:
dogs = np.asarray(dogs).astype('float32')/255.0
cats = np.asarray(cats).astype('float32')/255.0

In [15]:
dog_train = dogs[:10000]

In [16]:
cat_train = cats[:10000]

In [17]:
dog_test = dogs[10000:12500]
cat_test = cats[10000:12500]

In [18]:
dog_train.shape

(10000, 96, 96, 1)

In [19]:
#y_dogs = np.asarray(y_dogs[:10000]).astype('int32')
#y_cats = np.asarray(y_cats[:10000]).astype('int32')

In [20]:
y_dogs = np.asarray(y_dogs).astype('int32')
y_cats = np.asarray(y_cats).astype('int32')

In [21]:
#X_train = np.concatenate((dog_train, cat_train), axis=0)
#X_test =  np.concatenate((dog_test, cat_test), axis=0)

In [22]:
X_train = np.concatenate((dogs, cats), axis=0)
X_test =  np.concatenate((dogs, cats), axis=0)

In [23]:
y_train = np.concatenate((y_dogs, y_cats), axis=0)
y_test =  np.concatenate((y_dogs, y_cats), axis=0)

In [24]:
y_train.shape, y_test.shape

((25000,), (25000,))

In [25]:
y_train = tf.keras.utils.to_categorical(y_train, 2)
y_test = tf.keras.utils.to_categorical(y_test, 2)

In [26]:
model = Sequential([
    Conv2D(32, (3,3), padding='same', input_shape=(ROW, COL, 1), activation='relu'), #padding 사이즈 똑같이
    Conv2D(32, (3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.25),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)), # 사이즈가 반으로 줄어듦(48x48)
    Dropout(.25),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(.5),
    Dense(2, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0

In [27]:
model.compile(loss='categorical_crossentropy', optimizer =Adam(), metrics = ['accuracy'])

In [28]:
modelpath = 'model/dogs_vs_cats_full-cnn-{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath = modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [29]:
model.fit(X_train, y_train, batch_size=32, epochs =40, validation_split=0.2, callbacks=[checkpointer, early_stopping_callback])

Train on 20000 samples, validate on 5000 samples
Epoch 1/40
19968/20000 [============================>.] - ETA: 0s - loss: 0.6562 - accuracy: 0.6261
Epoch 00001: val_loss improved from inf to 0.70425, saving model to model/dogs_vs_cats_full-cnn-01-0.7043.hdf5
20000/20000 [==============================] - 311s 16ms/sample - loss: 0.6561 - accuracy: 0.6262 - val_loss: 0.7043 - val_accuracy: 0.4854
Epoch 2/40
19968/20000 [============================>.] - ETA: 0s - loss: 0.5882 - accuracy: 0.6850
Epoch 00002: val_loss did not improve from 0.70425
20000/20000 [==============================] - 310s 15ms/sample - loss: 0.5880 - accuracy: 0.6852 - val_loss: 0.7477 - val_accuracy: 0.5758
Epoch 3/40
19968/20000 [============================>.] - ETA: 0s - loss: 0.4834 - accuracy: 0.7698
Epoch 00003: val_loss did not improve from 0.70425
20000/20000 [==============================] - 311s 16ms/sample - loss: 0.4833 - accuracy: 0.7699 - val_loss: 0.9158 - val_accuracy: 0.5074
Epoch 4/40
19968/2

In [30]:
from tensorflow.keras.models import load_model
del model

In [33]:
model = load_model('model/dogs_vs_cats_full-cnn-05-0.4633.hdf5')

In [34]:
scores = model.evaluate(X_test, y_test, verbose=2)
print('MODEL ACCURACY: %.5f' % scores[1])

25000/25000 - 57s - loss: 0.3267 - accuracy: 0.8590
MODEL ACCURACY: 0.85900
